# Connect 4

In [1]:
import itertools
import numpy as np
import pandas as pd
from connectx import Connect4, wins_from_next_move

In [2]:
from platform import python_version
python_version()

'3.6.10'

In [3]:
game = Connect4()
game.name

'Connect 4'

In [4]:
game.make_move((1, 0))
game.show_state()

_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
X _ _ _ _ _ _


In [5]:
game.make_move((2, 4))
game.show_state()

_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
X _ _ _ O _ _


In [6]:
while True:
    game = Connect4()
    while not game.game_over:
        role = game.turn
        moves = game.available_moves()
        move = np.random.choice(moves)
        game.make_move((role, move))
    if game.winner is None:
        break

game.show_state()
print(game.winner)

O X O O O X X
X X O X X O O
O O X O O X X
X X X O X X O
O O X X O O O
X X O X O O X
None


In [7]:
game.moves

[(1, 6),
 (2, 6),
 (1, 0),
 (2, 2),
 (1, 1),
 (2, 6),
 (1, 6),
 (2, 6),
 (1, 3),
 (2, 5),
 (1, 2),
 (2, 5),
 (1, 3),
 (2, 1),
 (1, 1),
 (2, 3),
 (1, 5),
 (2, 0),
 (1, 6),
 (2, 3),
 (1, 5),
 (2, 5),
 (1, 5),
 (2, 4),
 (1, 0),
 (2, 1),
 (1, 2),
 (2, 0),
 (1, 2),
 (2, 2),
 (1, 1),
 (2, 2),
 (1, 1),
 (2, 4),
 (1, 4),
 (2, 4),
 (1, 4),
 (2, 4),
 (1, 3),
 (2, 3),
 (1, 0),
 (2, 0)]

### Test check_game_state methods

In [8]:
moves = [
            (1, 3), (2, 6), (1, 0), (2, 5), (1, 2),
            (2, 2), (1, 5), (2, 1), (1, 3), (2, 1),
            (1, 6), (2, 2), (1, 4), (2, 4), (1, 4),
            (2, 4), (1, 1), (2, 3)
        ]
game = Connect4(moves=moves)
game.show_state()

_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ O _ _
_ X O O X _ _
_ O O X O X X
X O X X X O O


In [9]:
game.check_game_state()

(True, 2)

In [10]:
%timeit game.check_game_state()

40 µs ± 672 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
# Game execution speed test
%timeit Connect4(moves=moves)

811 µs ± 29.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
# 3.06 ms with both checks
# 2.18 ms with full-state check
# 1.04 ms with checks on last move's position
# 807 µs with checks on last move's position and fast check for draw

In [13]:
# Test game with standard players
from gamelearner import GameController, RandomPlayer, GameController

game = Connect4()
player1 = RandomPlayer('R1')
player2 = RandomPlayer('R2')

In [14]:
players = [player1, player2]
ctrl = GameController(game, players)

In [15]:
ctrl.play(show=True)

Game of Connect 4 with 2 players ['R1', 'R2']
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
R1's turn (column from left): 1
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ X _ _ _ _ _
R2's turn (column from left): 0
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
O X _ _ _ _ _
R1's turn (column from left): 1
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ X _ _ _ _ _
O X _ _ _ _ _
R2's turn (column from left): 3
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ X _ _ _ _ _
O X _ O _ _ _
R1's turn (column from left): 2
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ X _ _ _ _ _
O X X O _ _ _
R2's turn (column from left): 1
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ O _ _ _ _ _
_ X _ _ _ _ _
O X X O _ _ _
R1's turn (column from left): 0
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ O _ _ _ _ _
X X _ _ _ _ _
O X X O _ _ _
R2's turn (column from left): 0
_ _ _ _ _ _ _
_ _ _ _ _ _ 

In [16]:
# Find winning moves from current state

def check_for_obvious_move(game, role, board_full=None, 
                           terminal_values={'win': 1, 'loss': -1, 'draw':0},
                           depth=1):
    """Analyses the current board state (or board_full if
    provided) from the perspective of the player role.
    
    Returns
        value, positions (float, list): value of the current
            state if it is a terminal state (1.0, 0.5 or -1.0, 
            else None), a list of best positions (columns) to 
            take in next move.
    """
    if board_full is None:
        board_full = game._board_full
        state = game.state
        fill_levels = game._fill_levels
    else:
        state = game.state_from_board_full(board_full)
        fill_levels = game._get_fill_levels(state)
    opponent = role ^ 3
    n_moves = fill_levels.sum()
    win_value, loss_value = (terminal_values['win'], 
                             terminal_values['loss'])

    # TODO: This needs to be not in this func
    # 0. Check if early move of game
    #if n_moves == 0:
    #    return None, [3]
    #elif (n_moves == 1) and (state[0,3] == opponent):
    #    return None, [3]
    
    # 1. Check for a win by role on next move
    possible_moves = wins_from_next_move(game, role, board_full=board_full)
    n_wins = sum(possible_moves.values())
    if n_wins > 0:
        winning_moves = [col for col, win in possible_moves.items() if win]
        return win_value, winning_moves

    # 2. Check if draw (last move but no win)
    if len(possible_moves) == 1:
        return terminal_values['draw'], possible_moves

    if depth > 0:
        # 3. Check what opponent could do next for each possible move
        bf2 = board_full.copy()  # TODO: Could remove if sure it is restored
        state = game.state_from_board_full(bf2)
        fill_levels = game._get_fill_levels(state)
        opp_wins = {}
        opp_losses = {}
        other_moves = []
        opp_move_values = {}
        for col in possible_moves:
            assert state[fill_levels[col], col] == 0  # TODO: delete later
            state[fill_levels[col], col] = role  # Next state after move
            value, moves = check_for_obvious_move(game, opponent, board_full=bf2,
                                                  depth=depth-1)
            opp_move_values[col] = value
            if value == win_value:
                opp_wins[col] = len(moves)
            elif value == terminal_values['loss']:
                opp_losses[col] = len(moves)
            else:
                other_moves.append(col)
            state[fill_levels[col], col] = 0  # Reverse move

        # 4. Take any move where opponent will definitely lose
        if len(opp_losses) > 0:
            return win_value, [col for col, value in opp_move_values.items()
                               if value == loss_value]

        # 5. If opponent will possibly win for all moves, assume defeat
        if len(opp_wins) == len(possible_moves):
            fewest = [col for col, n_win_moves in opp_wins.items()
                      if n_win_moves == min(opp_wins.values())]
            return loss_value, fewest

        # 6. Avoid any move where opponent will definitely win
        if len(opp_wins) > 0:
            return None, other_moves

    # Otherwise, return no value
    return None, list(possible_moves.keys())

In [17]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
])

role = 1
assert not any(
    wins_from_next_move(game, role, board_full=board_full).values()
)

# Best starting move
moves = check_for_obvious_move(game, role, board_full=board_full)
print(moves)
#assert moves == (None, [3])

game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
])

role = 2
assert not any(
    wins_from_next_move(game, role, board_full=board_full).values()
)

# Best response to best starting move
moves = check_for_obvious_move(game, role, board_full=board_full)
#assert moves == (None, [3])
print(moves)

(None, [0, 1, 2, 3, 4, 5, 6])
(None, [0, 1, 2, 3, 4, 5, 6])


In [18]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 1, 1, 1, 2, 2, 1],
    [0, 0, 1, 0, 2, 1, 0],
    [0, 0, 0, 0, 2, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 1, 0]
])

role = 1
moves = check_for_obvious_move(game, role, board_full=board_full)
assert moves == (1.0, [0])

role = 2
moves = check_for_obvious_move(game, role, board_full=board_full)
assert moves == (1.0, [4]) 

In [19]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 2, 1, 1, 2, 2, 1],
    [0, 0, 1, 0, 2, 1, 0],
    [0, 0, 0, 0, 2, 1, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 1, 0]
])

role = 1
# Should block win by player 2
moves = check_for_obvious_move(game, role, board_full=board_full)
assert moves == (None, [4])

In [20]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 1, 2, 1, 2, 2, 1],
    [0, 0, 1, 0, 2, 1, 0],
    [0, 0, 0, 0, 2, 1, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 1, 0]
])
role = 1
# Can't win but at least block one of moves
moves = check_for_obvious_move(game, role, board_full=board_full)
assert moves == (-1.0, [3, 4])

In [21]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 0, 1, 1, 2, 2, 1],
    [0, 0, 1, 2, 2, 1, 0],
    [0, 0, 0, 0, 2, 2, 0],
    [0, 0, 0, 0, 1, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 1, 0]
])

role = 1
assert not any(
    wins_from_next_move(game, role, board_full=board_full).values()
)
# This should identify the winning move (1)
moves = check_for_obvious_move(game, role, board_full=board_full, 
                               depth=2)
#assert moves == (1.0, [1])
moves

(1, [1])

In [22]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 0, 1, 1, 0, 0, 0],
    [0, 0, 0, 2, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
])

role = 2
assert not any(
    wins_from_next_move(game, role, board_full=board_full).values()
)

# TODO: This should identify a blocking move (1 or 4)
moves = check_for_obvious_move(game, role, board_full=board_full, 
                               depth=3)
assert moves == (None, [1, 4])

In [24]:
%timeit check_for_obvious_move(game, role, board_full=board_full, depth=3)

106 ms ± 9.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
STOP!!

SyntaxError: invalid syntax (<ipython-input-23-480d53eab50e>, line 1)

## Developing a game_state key

In [ ]:
shape = game.shape
fl = game._fill_levels
rows = np.empty((4, shape[1]), dtype='int8')
for i in range(4):
    rows[i] = (fl-i).clip(-1,)
rows

In [ ]:
cols = np.empty((4, shape[1]), dtype='int8')
cols[:] = np.arange(1,shape[1]+1)
cols

In [ ]:
board_full

In [ ]:
board_full[(rows, cols)]

In [ ]:
m_level = int(np.median(fl))
m_level

In [ ]:
state_key = np.empty((5, shape[1]), dtype='int8')
state_key[0, :] = (fl - m_level).clip(-1, 2)
state_key[1:, :] = board_full[(rows, cols)]
state_key

### Find all possible (valid) combinations of states for 4x4 segment

In [ ]:
from itertools import product

combinations = np.array(list(product(range(5), repeat=4))).sum(axis=1)
assert len(combinations) == 625

In [ ]:
n_discs = pd.Series(combinations).value_counts().sort_index().rename('No. of boards')
n_discs.index.name = 'No. of discs'
summary = pd.concat([
    n_discs, 
    pd.Series(2**n_discs.index.values, index=n_discs.index, name='Disc combinations')
], axis=1)
summary['Total combinations'] = summary['No. of boards'] * summary['Disc combinations']
summary

In [ ]:
summary['Total combinations'].sum()

### Generate a compact, hashable state representation

In [ ]:
a1 = np.array([64, 16, 4, 1])
a2 = np.array([16777216, 65536, 256, 1])

def generate_state_key_uint32(grid):
    """Convert 4x4 int8 array to int64 value.
    """
    assert grid.dtype == 'int8'
    assert grid.shape == (4, 4)
    return (np.sum(grid*a1, axis=1)*a2).sum().astype('uint32')

grid = 3*np.ones((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 2**32-1
assert generate_state_key_uint32(grid).dtype == 'uint32'
assert (generate_state_key_uint32(grid) + 1).dtype == 'int64'
grid = np.zeros((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 0
grid[:, 3] = 1
assert generate_state_key_uint32(grid) == 1 + 256 + 256**2 + 256**3
grid = np.ones((4,3), dtype='int8') 
try:
    generate_state_key_uint32(grid)
except AssertionError:
    pass

grid = np.random.choice([0, 1, 2], size=16).astype('int8').reshape((4,4))
print(grid)
generate_state_key_uint32(grid)

In [ ]:
# sum([30.9, 31, 31.1, 28.9])/4 == 30.5
%timeit generate_state_key_uint32(grid)

In [ ]:
a1 = np.array([6, 4, 2, 0])
a2 = np.array([24, 16, 8, 0])

def generate_state_key_uint32(grid):
    """Convert 4x4 int8 array to int64 value.
    """
    assert grid.dtype == 'int8'
    return np.sum(np.sum(grid << a1, axis=1) << a2).astype('uint32')

grid = 3*np.ones((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 2**32-1
assert generate_state_key_uint32(grid).dtype == 'uint32'
assert (generate_state_key_uint32(grid) + 1).dtype == 'int64'
grid = np.zeros((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 0
grid[:, 3] = 1
assert generate_state_key_uint32(grid) == 1 + 256 + 256**2 + 256**3
grid = np.ones((4,3), dtype='int8') 
try:
    generate_state_key_uint32(grid)
except ValueError:
    pass

grid = np.random.choice([0, 1, 2], size=16).astype('int8').reshape((4,4))
print(grid)
generate_state_key_uint32(grid)

In [ ]:
# sum([31.2, 31.9, 31.8, 32.4])/4 == 31.8
%timeit generate_state_key_uint32(grid)

In [ ]:
a = np.array([1, 2, 3, 0], dtype='int8')

v = np.sum(a << np.array([6, 4, 2, 0])).astype('uint32')

v

In [ ]:
(v >> 6) & 3, (v >> 4) & 3, (v >> 2) & 3, v & 3

In [ ]:
np.array([v >> 6, v >> 4, v >> 2, v], dtype='int8') & 3